In [ ]:
!pip install deap

In [1]:

import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


### Check Sample example of : a<sup>3</sup> - b<sup>2</sup>  + c - d + 2e

In [2]:

# Define new functions
def protectedDiv(left, right):
    with numpy.errstate(divide='ignore',invalid='ignore'):
        x = numpy.divide(left, right)
        if isinstance(x, numpy.ndarray):
            x[numpy.isinf(x)] = 1
            x[numpy.isnan(x)] = 1
        elif numpy.isinf(x) or numpy.isnan(x):
            x = 1
    return x

pset = gp.PrimitiveSet("MAIN", 5)
pset.addPrimitive(numpy.add, 2, name="vadd")
pset.addPrimitive(numpy.subtract, 2, name="vsub")
pset.addPrimitive(numpy.multiply, 2, name="vmul")
pset.addPrimitive(protectedDiv, 2)
pset.renameArguments(ARG0='a', ARG1='b', ARG2='c', ARG3='d', ARG4='e')

creator.create("FitnessMin2", base.Fitness, weights=(-1.0,))
creator.create("Individual2", gp.PrimitiveTree, fitness=creator.FitnessMin2)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual2, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

samples = numpy.random.uniform(size=(10, 5))
values = samples[:,0]**3 + samples[:,1]**2 + samples[:,2] - samples[:,3] + samples[:,4]*2

def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the sum of squared difference between the expression
    diff = numpy.sum((func(*samples.T) - values)**2)
    return diff,

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def function2():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats, halloffame=hof)

    return pop, stats, hof


In [3]:
pop, stats, hof = function2()

gen	nevals	avg    	std    	min    	max  
0  	300   	228.192	1673.77	1.59983	20538
1  	169   	105.258	1450.14	1.59983	25152.1
2  	161   	15.5714	54.0321	0.774496	607.169
3  	193   	18.4205	201.609	0.774496	3490.18
4  	170   	4.52395	14.4987	0.774496	218.63 
5  	176   	62.0438	712.347	0.774496	11321.8
6  	157   	14.7867	136.426	0.774496	2086.87
7  	145   	2.47963	8.93642	0.774496	147.461
8  	170   	191.11 	2462.69	0.774496	39570.5
9  	171   	2.9317 	8.91558	0.774496	145.46 
10 	152   	3.51902	17.2363	0.614599	243.799
11 	177   	246.988	4122.39	0.614599	71520.3
12 	177   	3.06261	14.4435	0.614599	245.646
13 	157   	228.145	3906.81	0.614599	67783.3
14 	176   	4.00728	16.309 	0.614599	210.076
15 	185   	76.9871	1256.33	0.614599	21798.9
16 	165   	468.441	8048.69	0.774496	139643 
17 	139   	2.9249 	15.9851	0.76818 	274.321
18 	155   	3.17066	22.4332	0.76818 	387.821
19 	173   	5.65811	59.0551	0.6773  	1025.26
20 	168   	3.72166	22.5065	0.6773  	385.84 
21 	172   	4.46142	24.7407	0.6773  	362

In [4]:
for data in [ hof]:
  for ind in data:
      print(ind)

vadd(vadd(c, vmul(vadd(c, vadd(a, vmul(vmul(vadd(c, c), vmul(e, e)), c))), vmul(e, e))), e)
